## SEQUENCE CLASSIFICATION

In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
import tensorflow.contrib.rnn as rnn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
print ("PACKAGES LOADED")

MNIST = input_data.read_data_sets("data/", one_hot=True)
trainimgs, trainlabels, testimgs, testlabels \
 = MNIST.train.images, MNIST.train.labels, MNIST.test.images, MNIST.test.labels 
ntrain, ntest, dim, nclasses \
 = trainimgs.shape[0], testimgs.shape[0], trainimgs.shape[1], trainlabels.shape[1]
print ("MNIST LOADED")
print ("TF VERSION %s" % (tf.__version__))

PACKAGES LOADED
Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIST LOADED
TF VERSION 1.0.1


## THIS IS HOW RNN WILL WORK

<img src="data/1.jpg" width="600"/>
<img src="data/2.jpg" width="600"/> 

## DEFINE MODEL

In [2]:
diminput  = 28
dimhidden = 128
dimoutput = nclasses
nsteps    = 28
weights = {
    'hidden': tf.Variable(tf.random_normal([diminput, dimhidden])), 
    'out': tf.Variable(tf.random_normal([dimhidden, dimoutput]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([dimhidden])),
    'out': tf.Variable(tf.random_normal([dimoutput]))
}
def _RNN(_X, _istate, _W, _b, _nsteps, _name):
    # 1. PERMUTE INPUT FROM [batchsize, nsteps, diminput] TO [nsteps, batchsize, diminput]
    _X = tf.transpose(_X, [1, 0, 2])
    # 2. RESHAPE INPUT TO [nsteps*batchsize, diminput] 
    _X = tf.reshape(_X, [-1, diminput])
    # 3. INPUT TO HIDDEN LAYER
    _H = tf.matmul(_X, _W['hidden']) + _b['hidden']
    # 4. SPLIT DATA TO 'NSTEPS' CHUNKS
    _Hsplit = tf.split(_H, _nsteps, axis=0) 
    # 5. GET LSTM'S FINAL OUTPUT (_LSTM_O) AND STATE (_LSTM_S)
    with tf.variable_scope(_name):
        lstm_cell = rnn.BasicLSTMCell(dimhidden, forget_bias=1.0
                                        , state_is_tuple=False)
        _LSTM_O, _LSTM_S = rnn.static_rnn(lstm_cell
                                        , _Hsplit, initial_state=_istate)
    # OUTPUT
    _O = tf.matmul(_LSTM_O[-1], _W['out']) + _b['out']    
    return {
        'X': _X, 'H': _H, 'Hsplit': _Hsplit,
        'LSTM_O': _LSTM_O, 'LSTM_S': _LSTM_S, 'O': _O 
    }
print ("Network ready")

Network ready


## DEFINE GRAPH

In [3]:
learning_rate = 0.001
x      = tf.placeholder("float", [None, nsteps, diminput])
istate = tf.placeholder("float", [None, 2*dimhidden]) 
    # state & cell => 2x n_hidden
y      = tf.placeholder("float", [None, dimoutput])
myrnn  = _RNN(x, istate, weights, biases, nsteps, 'basic')
pred   = myrnn['O']
cost   = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y)) 
optm   = tf.train.AdamOptimizer(learning_rate).minimize(cost) 
accr   = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred,1), tf.argmax(y,1)), tf.float32))
init   = tf.global_variables_initializer()
print ("NETWORK READY")

NETWORK READY


## RUN

In [4]:
training_epochs = 5
batch_size      = 128
display_step    = 1
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
sess.run(init)
print ("START OPTIMIZATION.")
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(MNIST.train.num_examples/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_xs, batch_ys = MNIST.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, nsteps, diminput))
        # Fit training using batch data
        feeds = {x: batch_xs, y: batch_ys, istate: np.zeros((batch_size, 2*dimhidden))}
        sess.run(optm, feed_dict=feeds)
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict=feeds)/total_batch
    # Display logs per epoch step
    if epoch % display_step == 0: 
        print ("EPOCH: %03d/%03d COST: %.9f" % (epoch, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys, istate: np.zeros((batch_size, 2*dimhidden))}
        train_acc = sess.run(accr, feed_dict=feeds)
        print (" TRAIN ACCURACY: %.3f" % (train_acc))
        testimgs = testimgs.reshape((ntest, nsteps, diminput))
        feeds = {x: testimgs, y: testlabels, istate: np.zeros((ntest, 2*dimhidden))}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("  TEST ACCURACY: %.3f" % (test_acc))
print ("OPTIMIZATION FINISHED.")

START OPTIMIZATION.
EPOCH: 000/005 COST: 0.580071183
 TRAIN ACCURACY: 0.984
  TEST ACCURACY: 0.878
EPOCH: 001/005 COST: 0.148037395
 TRAIN ACCURACY: 0.977
  TEST ACCURACY: 0.960
EPOCH: 002/005 COST: 0.091153915
 TRAIN ACCURACY: 0.945
  TEST ACCURACY: 0.972
EPOCH: 003/005 COST: 0.067590098
 TRAIN ACCURACY: 0.969
  TEST ACCURACY: 0.973
EPOCH: 004/005 COST: 0.054751573
 TRAIN ACCURACY: 0.969
  TEST ACCURACY: 0.977
OPTIMIZATION FINISHED.


## WHAT IF WE USE SMALLER NUMBER OF SEQS?

In [5]:
for _nsteps in [24, 25, 26, 27, 28]:
    # TEST WITH TRUNCATED SEQS
    testimgs = testimgs.reshape((ntest, nsteps, diminput))
    testimgs_trucated = np.zeros(testimgs.shape)
    testimgs_trucated[:, 28-_nsteps:] = testimgs[:, :_nsteps, :]
    feeds = {x: testimgs_trucated, y: testlabels, istate: np.zeros((ntest, 2*dimhidden))}
    test_acc = sess.run(accr, feed_dict=feeds)
    print (" WITH [%d] SEQS, TEST ACCR IS [%.3f]" 
           % (_nsteps, test_acc))

 WITH [24] SEQS, TEST ACCR IS [0.656]
 WITH [25] SEQS, TEST ACCR IS [0.830]
 WITH [26] SEQS, TEST ACCR IS [0.941]
 WITH [27] SEQS, TEST ACCR IS [0.972]
 WITH [28] SEQS, TEST ACCR IS [0.977]


## INSIDE RNN

### INPUTS TO THE RNN

In [6]:
batch_size = 5
xtest, _ = MNIST.test.next_batch(batch_size)
print ("Shape of 'xtest' is %s" % (xtest.shape,))

Shape of 'xtest' is (5, 784)


### RESHAPED INPUTS

In [7]:
xtest1 = xtest.reshape((batch_size, nsteps, diminput))
print ("Shape of 'xtest1' is %s" % (xtest1.shape,))

Shape of 'xtest1' is (5, 28, 28)


### FEEDS

In [8]:
feeds = {x: xtest1, istate: np.zeros((batch_size, 2*dimhidden))}

### INDIVISUAL INPUT TO THE LSTM

In [9]:
rnnout_X = sess.run(myrnn['X'], feed_dict=feeds)
print ("Shape of 'rnnout_X' is %s" % (rnnout_X.shape,))

Shape of 'rnnout_X' is (140, 28)


### INTERMEDIATE STATE

In [10]:
rnnout_H = sess.run(myrnn['H'], feed_dict=feeds)
print ("Shape of 'rnnout_H' is %s" % (rnnout_H.shape,))

Shape of 'rnnout_H' is (140, 128)


### ACTUAL INPUT TO LSTSM

In [11]:
rnnout_Hsplit = sess.run(myrnn['Hsplit'], feed_dict=feeds)
print ("Type of 'rnnout_Hsplit' is %s" % (type(rnnout_Hsplit)))
print ("Length of 'rnnout_Hsplit' is %s and the shape of each item is %s" 
       % (len(rnnout_Hsplit), rnnout_Hsplit[0].shape))

Type of 'rnnout_Hsplit' is <type 'list'>
Length of 'rnnout_Hsplit' is 28 and the shape of each item is (5, 128)


### OUTPUT FROM THE LSTM

In [12]:
rnnout_LSTM_O = sess.run(myrnn['LSTM_O'], feed_dict=feeds)
print ("Type of 'rnnout_LSTM_O' is %s" % (type(rnnout_LSTM_O)))
print ("Length of 'rnnout_LSTM_O' is %s and the shape of each item is %s" 
       % (len(rnnout_LSTM_O), rnnout_LSTM_O[0].shape))

Type of 'rnnout_LSTM_O' is <type 'list'>
Length of 'rnnout_LSTM_O' is 28 and the shape of each item is (5, 128)


### FINAL PREDICTION

In [13]:
rnnout_O = sess.run(myrnn['O'], feed_dict=feeds)
print ("Shape of 'rnnout_O' is %s" % (rnnout_O.shape,))

Shape of 'rnnout_O' is (5, 10)
